## The Monty Hall Problem

The Monty Hall problem is a puzzle loosely based on the TV game show *Let's Make a Deal* and seems to be counterintuitive for many people.  The problem setup is straightforward enough:  Suppose you are a contestant in a game show, and the host Monty Hall shows you 3 doors.  One of the doors has a prize behind it and the other 2 doors have goats behind them.  Monty tells you to pick a door.  After you've picked a door, Monty does something gnarly:  He opens one of the 2 remaining doors and reveals a goat.  Then he asks you if you want to switch doors or stick with your remaining choice.  Obviously we are assuming that you would rather have the prize and not the goat.  The question is:  Should you switch or not?

Many people think that it doesn't matter whether you switch or not, and they reason that there should be an equal probability ($\,=\frac{1}{2}$) that the prize is behind the door you chose versus the door that Monty Hall didn't open.  In fact, the optimal solution is to always switch doors.  Switching doors give you a $\frac{2}{3}$ probability of getting the prize, while not switching gives you a $\frac{1}{3}$ probability.  This tends to surprise people, and some people will even refuse to accept that switching is the best strategy.  There seem to be some [interesting psychological reasons](http://www.diss.fu-berlin.de/diss/servlets/MCRFileNodeServlet/FUDISS_derivate_000000001141/04_chap2.pdf) for this.

But I don't want to get into the psychology of the Monty Hall problem.  I want to first give a brief explanation of why the optimal strategy is switching, then I want to code up the problem using Python and simulate lots and lots of Monty Hall games (say, 100000 games).  By this we can estimate the true probability of winning the prize by always switching and we will see that the proportion of wins will be very close to $\frac{2}{3}$ with the switching strategy and $\frac{1}{3}$ otherwise.  Many other people have simulated the Monty Hall game as well.  Here is a [YouTube video](https://www.youtube.com/watch?v=nwpqkb92GYw&feature=youtu.be) that describes a nice objected-oriented approach, where they model the problem with a Python class.

After running the simulation, I want to give a proof of the Monty Hall problem using a bit of probability theory.  The proof is not new, and is similar to the one using Bayes' theorem on the [wikipedia page](https://en.wikipedia.org/wiki/Monty_Hall_problem).

### 1. An intuitive explanation

First let's give a relatively simple explanation why the optimal strategy in the Monty Hall problem is always switching from the intitial guess.  The steps in the problem are as follows:
1. The contestant chooses a door randomly.
2. Monty Hall opens one of the remaining 2 doors and reveals a goat and not the prize.  
3. The contestant is given the choice whether to switch doors or stay with their original guess.


#### Case 1:  The contestant initially chooses the door with the prize ( probability = 1/3 )
- In this case the 2 remaining doors both have goats, so Monty will pick one of them and reveal a goat.  If the contestant switches, they don't win the prize.  If they stick with their original guess, then they do win the prize.  So in this case it's better not to switch, *but this only happens with probability 1/3*.

#### Case 2:  The contestant initially chooses the door with a goat ( probability = 2/3 )
- In this case, one of the remaining doors has a goat and one of them has the prize.  *Monty must open the door with the goat, so in this case we are certain that the prize is behind the remaining door*.  Therefore, if the contestant switches, they win the prize.  If they don't switch, they don't win the prize.  This always happens with probability $\frac{2}{3}$, so it is always better to switch the initial guess.  

In the long run (after playing many Monty Hall games), we should expect that switching the initial guess will lead to the contestants winning prize approximately 2/3 of the time (~ 67%), and those who don't switch should win the prize approximately 1/3 of the time (~ 33%).  This is a consequence of the [law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers).  Let's simulate this now.

### 2.  Simulating the Monty Hall games

(more explanation to follow...)

In [1]:
import numpy as np
import random

In [2]:
def prizedoor(n):
    '''Given a positive integer n, return a random array of length n
    where each entry is either 0, 1, or 2 representing a hidden 
    prize behind door 0, 1, or 2.  Run n simulations of the prizedoor.'''
    
    return np.random.randint(3, size = n)

def initial_guess(n):
    '''This function also returns a random array of length n consisting
    of 0s, 1s, and 2s representing n simulations of contestants guessing
    a door randomly.'''
    
    return np.random.randint(3, size = n)


Goat door

In [3]:
def goatdoor(prizedoors, guesses):
    '''This function simulates opening a "goat door" for each prize door
    and each guess.  The goat door should not be the prize door and should 
    be different from the contestant's guess.  Returns an array of goatdoors
    for n Monty Hall games.'''
    
    goatdoors = []
    # loop over all pairs of prizedoors and guesses
    for pair in zip(prizedoors, guesses):
        if pair[0] == pair[1]:
            # if the contestant guesses the prizedoor, Monty randomly picks one of the two remaining doors 
            other_doors = [ d for d in [0,1,2] if d != pair[0] ]
            goatdoors.append( random.choice(other_doors) )
        else:
            # if the guess is different from the prizedoor, Monty picks the one remaining door
            other_door = [ d for d in [0,1,2] if d not in (pair[0], pair[1]) ]
            goatdoors.append( other_door[0] )
        
    return np.array(goatdoors)

Switch guess

In [4]:
def switch_guess(guesses, goatdoors):
    '''This function implements the strategy of always switching
    a guess after a goat door is opened.  Returns an array of length
    n representing the contestants switching doors in n Monty Hall games.'''
    
    switches = []
    # loop over all pairs of guesses and goatdoors
    for pair in zip(guesses, goatdoors):
        # the contestant only has one choice of door to switch to after the goatdoor is opened
        switch_door = [ d for d in [0,1,2] if d not in (pair[0], pair[1]) ]
        switches.append( switch_door[0] )
        
    return np.array(switches)
    

Now let's simulate the game both with the strategy of switching and without the strategy of switching

In [5]:
def run_monty(n, switch=True):
    '''Simulate n Monty Hall games.  The default value of switch
    represents the contestants always switching their initial guess.
    Set switch=False to simulate the contestants not switching.'''
    
    prizedoors = prizedoor(n)
    guesses = initial_guess(n)
    goatdoors = goatdoor(prizedoors, guesses)
    if switch:
        contestantdoors = switch_guess(guesses, goatdoors)
    else:
        contestantdoors = guesses
        
    wins = 0
    # loop over all pairs of prizedoors and contestantdoors
    for pair in zip(prizedoors, contestantdoors):
        # if a prizedoor is the same as the corresponding contestantdoor, then
        # the contestant wins
        if pair[0] == pair[1]:
            wins += 1
            
    # print the relevant stats
    print('  Number of games: ', n)
    print('  Number of wins:  ', wins)
    print('  Win Percentage:   %.2f% %' % round(100 * wins / n, 2) )

print('Monty Hall simulations with switching the initial guess: ')
run_monty(100000, switch=True)
print()
print('Monty Hall simulations with not switching the initial guess: ')
run_monty(100000, switch=False)

Monty Hall simulations with switching the initial guess: 
  Number of games:  100000
  Number of wins:   66464
  Win Percentage:   66.46%

Monty Hall simulations with not switching the initial guess: 
  Number of games:  100000
  Number of wins:   33370
  Win Percentage:   33.37%


### 3.  A mathematical proof

...